In [1]:
import torch
from torch import nn

torch.__version__


'2.6.0'

In [2]:
# setup device agonostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

### GET DATA

In [22]:
import requests
import zipfile
from pathlib import Path

# setup data path
data_path = Path("data")
image_path = data_path / "pizza_steak_sushi"
image_path

# if data already exist
if image_path.is_dir():
    print("Path already exist... skipping download")
else:
    image_path.mkdir(parents=True, exist_ok=True)
    with open(image_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading data")
        f.write(request.content)
        

# unizip the content
with zipfile.ZipFile(image_path/"pizza_steak_sushi.zip", "r") as f:
    print("Unzippimg the file")
    f.extractall(image_path)




Unzippimg the file


In [23]:
import os
def walk_dir(path):
    """Walk in the path of dir"""
    for dirpath, dirnames, filenames in  os.walk(path):
        print(f"There are {len(dirnames)}directories and {len(filenames)} images in {dirpath}")
    

In [24]:
walk_dir(image_path)

There are 2directories and 1 images in data/pizza_steak_sushi
There are 3directories and 0 images in data/pizza_steak_sushi/test
There are 0directories and 19 images in data/pizza_steak_sushi/test/steak
There are 0directories and 31 images in data/pizza_steak_sushi/test/sushi
There are 0directories and 25 images in data/pizza_steak_sushi/test/pizza
There are 3directories and 0 images in data/pizza_steak_sushi/train
There are 0directories and 75 images in data/pizza_steak_sushi/train/steak
There are 0directories and 72 images in data/pizza_steak_sushi/train/sushi
There are 0directories and 78 images in data/pizza_steak_sushi/train/pizza


In [26]:
train_dir = image_path/"train"
test_dir = image_path/"test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))